# Importing Packages

In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
from scipy.special import jv
from scipy.integrate import quad  
from scipy.special import j0, j1
import tkinter as tk 
from tkintertable.Tables import TableCanvas
from tkintertable.TableModels import TableModel
from tkinter import ttk
from tkinter import filedialog 
#from compute import Calculation
from WuWanv02 import Calculation
#from compute_cython import Calculation
#from backcompute import Back_Calculation
from tkinter import *  
import cProfile
import pstats
# python setup.py build_ext --inplace
# python setup.py build_ext --inplace --compiler=mingw32

In [2]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import time
from tkintertable import TableCanvas, TableModel

class TableApp:
    def __init__(self, root):
        self.frame = ttk.Frame(root)
        self.frame.pack(fill=tk.BOTH, expand=True)

        # 创建 TableModel 与 TableCanvas
        self.model = TableModel()
        self.table = TableCanvas(self.frame, model=self.model, rowheight=30)
        self.table.show()

        # 载入初始数据
        self.load_data()
        # 调整列宽
        self.adjust_column_widths()
        # 设定初始颜色
        self.set_cell_colors()

        # 计算按钮
        button_frame = ttk.Frame(root)
        button_frame.pack(pady=10)
        tk.Button(
            button_frame,
            text="Compute Deflections",
            command=self.get_data,
            font=("Arial", 10, "bold"),
            bg="#4CAF50",
            fg="white",
            padx=20,
            pady=5
        ).pack()

        # 结果输出框
        self.result_text = tk.Text(root, height=5, width=200)
        self.result_text.pack(pady=10)

    # ------------------------------------------------- 数据与表格 -------------------------------------------------
    def load_data(self):
        data = {
            'Layer':            ['layer 1', 'layer 2', 'layer 3', 'layer 4', 'layer 5', '', ''],
            'Modulus [MPa]':    [4000, 400, 300, 200, 100, '', ''],
            'Poisson [-]':      ['0.3', '0.35', '0.35', '0.4', '0.4', '', ''],
            'Thickness [mm]':   ['150', '240', '300', '500', 'inf', '', ''],
            'Load':             ['load 1', '', '', '', '', '', ''],
            'Stress [MPa]':     ['0.95', '', '', '', '', '', ''],
            'Radius [mm]':      ['150', '', '', '', '', '', ''],
            'Evaluation point': ['point 1', 'point 2', 'point 3', 'point 4', 'point 5', 'point 6', 'point 7'],
            'r [mm]':           ['0', '300', '600', '900', '1200', '1500', '1800'],
            'Deflection [μm]':  ['', '', '', '', '', '', ''],
            'perfect: 1, no: 0': ['1', '', '', '', '', '', ''],
        }
        df = pd.DataFrame(data)

        # 清空再导入
        self.model.data = {}
        self.model.columnNames = []
        self.model.reclist = []
        self.model.importDict(df.to_dict(orient='index'))
        self.table.redraw()

    def adjust_column_widths(self):
        for col in self.model.columnNames:
            self.model.columnwidths[col] = max(len(col) * 10, 90)
        self.table.redraw()

    # ------------------------------- 颜色相关 -------------------------------
    def col_idx(self, col_name):
        """由列名取列索引；不存在返回 None"""
        try:
            return self.model.columnNames.index(col_name)
        except ValueError:
            return None

    def color_cells(self, rows, cols, color):
        """包装一层，保证 rows/cols 为列表且传 key='bg'"""
        if rows is not None and not isinstance(rows, (list, tuple)):
            rows = [rows]
        if cols is not None and not isinstance(cols, (list, tuple)):
            cols = [cols]
        self.table.setcellColor(rows, cols, color, key='bg')

    def set_cell_colors(self):
        """为说明文字单元格涂灰；空串或纯数字保持默认；结果列淡红"""
        grey   = '#7E9BB7'   # 说明文字背景
        #result = '#D87659'   # 结果列预设色

        # 需要检测的文字说明列
        cols_text = {
            'Layer',
            'Load',
            'Evaluation point',
            'perfect: 1, no: 0'
        }

        # 把列名换成索引，过滤掉不存在的列
        text_col_idx = [self.col_idx(c) for c in cols_text if self.col_idx(c) is not None]
        defl_col     = self.col_idx('Deflection [μm]')

        # 辅助：判定一个值是否包含字母
        def has_alpha(val):
            if not isinstance(val, str):
                return False
            return any(ch.isalpha() for ch in val)

        # 遍历行、列，只给包含字母的单元格上灰色
        for row, rec in self.model.data.items():          # rec 是该行的 dict
            for col in text_col_idx:
                col_name = self.model.columnNames[col]
                if has_alpha(rec.get(col_name, '')):
                    self.color_cells(row, col, grey)

        # 结果列整列预着色（想要只高亮已计算的可保留 highlight_results）
        #if defl_col is not None:
            #rows_all = list(self.model.data.keys())
            #self.color_cells(rows_all, defl_col, result)

        self.table.redraw()


    def highlight_results(self):
        """计算完成后把已填数字的 Deflection [μm] 再高亮"""
        deflect_col = self.col_idx('Deflection [μm]')
        if deflect_col is None:
            return
        rows_to_hl = []
        for r in range(7):
            v = str(self.model.data.get(r, {}).get('Deflection [μm]', ''))
            if v not in ('', 'nan', 'None'):
                rows_to_hl.append(r)
        if rows_to_hl:
            self.color_cells(rows_to_hl, deflect_col, '#D6AFB9')  
        self.table.redraw()

    # -------------------------------- 计算按钮 --------------------------------
    def get_data(self):
        df = pd.DataFrame.from_dict(self.model.data, orient='index')
        df.to_csv('data.csv', index=False)

        df_num = df.apply(pd.to_numeric, errors='coerce').fillna(0.0)
        arr = df_num.to_numpy()

        t0 = time.time()
        _, u1, u2, u3, u4, u5, u6, u7 = Calculation(arr)
        values_mm  = [u1, u2, u3, u4, u5, u6, u7]
        values_um  = [round(v * 1000, 3) for v in values_mm]

        df.loc[:6, 'Deflection [mm]'] = values_mm
        df.loc[:6, 'Deflection [μm]'] = values_um
        self.model.data = df.to_dict(orient='index')

        self.highlight_results()

        self.result_text.insert(tk.END,
                                f'Compute Finished!  Time: {time.time()-t0:.3f} s\n')
        print("Compute Finished!")

# -------------------------------- 主程序 --------------------------------
if __name__ == '__main__':
    root = tk.Tk()
    root.geometry('1280x600')
    root.title('WuWan v0.2')

    app = TableApp(root)
    root.mainloop()


Run time: 0.000138900 s
Compute Finished!
Run time: 0.000158900 s
Compute Finished!
Run time: 0.000114100 s
Compute Finished!
Run time: 0.000112600 s
Compute Finished!
Run time: 0.000134400 s
Compute Finished!
Run time: 0.000112200 s
Compute Finished!
Run time: 0.000121400 s
Compute Finished!
Run time: 0.000137700 s
Compute Finished!
Run time: 0.000118500 s
Compute Finished!
Run time: 0.000152900 s
Compute Finished!
Run time: 0.000120700 s
Compute Finished!


In [3]:
class TableApp:
    def __init__(self, root):
        self.frame = ttk.Frame(root)
        self.frame.pack(fill=tk.BOTH, expand=True)

        # Create a TableModel and TableCanvas
        self.model = TableModel()
        self.table = TableCanvas(self.frame, model=self.model, rowheight=30)
        self.table.show()

        # Load initial data
        self.load_data()

        # Adjust column widths
        self.adjust_column_widths()
        
        # **Set cell colors after loading data**
        self.set_cell_colors()

        # Create a frame for the button to center it
        button_frame = ttk.Frame(root)
        button_frame.pack(pady=10)
        
        # Only keep the Compute Deflections button
        self.get_data_button1 = tk.Button(
            button_frame, 
            text="Compute Deflections", 
            command=self.get_data,
            font=("Arial", 10, "bold"),
            bg="#4CAF50",
            fg="white",
            padx=20,
            pady=5
        )
        self.get_data_button1.pack()

        # Create the result Text widget
        self.result_text = tk.Text(root, height=5, width=200)
        self.result_text.pack(pady=10)

    def load_data(self):
        # Sample pandas DataFrame
        data = {
            'Layer': ['layer 1', 'layer 2', 'layer 3', 'layer 4', 'layer 5', '', ''],
            'Modulus [MPa]': [4000, 400, 300, 200, 100, '', ''],
            'Poisson [-]': ['0.3', '0.35', '0.35', '0.4', '0.4', '', ''],
            'Thickness [mm]': ['150', '240', '300', '500', 'inf', '', ''],
            'Load': ['load_1', '', '', '', '', '', ''],
            'Stress [MPa]': ['0.95', '', '', '', '', '', ''],
            'Radius [mm]': ['150', '', '', '', '', '', ''],
            'Evaluation point': ['point 1', 'point 2', 'point 3', 'point 4', 'point 5', 'point 6', 'point 7'],
            'r [mm]': ['0', '300', '600', '900', '1200', '1500', '1800'],
            'Deflection [μm]': ['', '', '', '', '', '', ''],
            'perfect: 1, no: 0': ['1', '', '', '', '', '', ''],
        }
        df = pd.DataFrame(data)

        # Clear the model's data and column names
        self.model.data = {}
        self.model.columnNames = []
        self.model.reclist = []

        # Convert DataFrame to dict and import to TableModel
        data_dict = df.to_dict(orient='index')
        self.model.importDict(data_dict)
        self.table.redraw()

    def adjust_column_widths(self):
        # Adjust the column widths based on column headers and content length
        for col_name in self.model.columnNames:
            self.model.columnwidths[col_name] = max(len(col_name) * 10, 80)
        self.table.redraw()

    def get_column_index(self, column_name):
        """**Get column index by column name**"""
        try:
            return self.model.columnNames.index(column_name)
        except ValueError:
            return None

    def set_cell_colors(self):
        """**Set different background colors for specific cells**"""
        
        # **Define color scheme**
        colors = {
            'header': '#E6F3FF',           # Light blue for headers
            'layer': ['#FFE6E6', '#E6FFE6', '#FFFFE6', '#F0E6FF', '#FFE6F0'],  # Different pastel colors for layers
            'load': '#FFF2E6',             # Light orange for load data
            'evaluation': '#E6F7FF',       # Light cyan for evaluation points
            'result': '#F0FFF0',           # Light green for results
        }
        
        # **Get column indices**
        layer_col = self.get_column_index('Layer')
        modulus_col = self.get_column_index('Modulus [MPa]')
        poisson_col = self.get_column_index('Poisson [-]')
        thickness_col = self.get_column_index('Thickness [mm]')
        load_col = self.get_column_index('Load')
        stress_col = self.get_column_index('Stress [MPa]')
        radius_col = self.get_column_index('Radius [mm]')
        eval_point_col = self.get_column_index('Evaluation point')
        r_col = self.get_column_index('r [mm]')
        deflection_col = self.get_column_index('Deflection [μm]')
        perfect_col = self.get_column_index('perfect: 1, no: 0')
        
        # **Set colors for material layers (rows 0-4)**
        for row in range(5):  # rows 0-4 for layers 1-5
            if row < len(colors['layer']):
                layer_color = colors['layer'][row]
                # **Color the material property columns**
                if layer_col is not None:
                    self.table.setcellColor(row, layer_col, layer_color)
                if modulus_col is not None:
                    self.table.setcellColor(row, modulus_col, layer_color)
                if poisson_col is not None:
                    self.table.setcellColor(row, poisson_col, layer_color)
                if thickness_col is not None:
                    self.table.setcellColor(row, thickness_col, layer_color)
        
        # **Set colors for load section (row 0)**
        load_color = colors['load']
        if load_col is not None:
            self.table.setcellColor(0, load_col, load_color)
        if stress_col is not None:
            self.table.setcellColor(0, stress_col, load_color)
        if radius_col is not None:
            self.table.setcellColor(0, radius_col, load_color)
        if perfect_col is not None:
            self.table.setcellColor(0, perfect_col, load_color)
        
        # **Set colors for evaluation points (all 7 rows)**
        eval_color = colors['evaluation']
        for row in range(7):  # 7 evaluation points
            if eval_point_col is not None:
                self.table.setcellColor(row, eval_point_col, eval_color)
            if r_col is not None:
                self.table.setcellColor(row, r_col, eval_color)
        
        # **Set colors for results column**
        result_color = colors['result']
        if deflection_col is not None:
            for row in range(7):
                self.table.setcellColor(row, deflection_col, result_color)
        
        # **Redraw the table to apply colors**
        self.table.redraw()

    def get_data(self):
        # Access the data from the model
        data = self.model.data
              
        # Convert it back to pandas DataFrame
        df = pd.DataFrame.from_dict(data, orient='index')

        # Save
        df.to_csv('data.csv', index=False)
        
        # Run compute module
        df_numeric = df.apply(pd.to_numeric, errors='coerce')
        df_numeric_filled = df_numeric.fillna(0).astype(float)
        Data_array = df_numeric_filled.to_numpy()
        start_time = time.time()
        
        resultz, resultu1, resultu2, resultu3, resultu4, resultu5, resultu6, resultu7 = Calculation(Data_array)

        result_values = [resultu1, resultu2, resultu3, resultu4, resultu5, resultu6, resultu7]
        result_values_microns = [round(value * 1000, 3) for value in result_values]
    
        df.loc[:len(result_values_microns)-1, 'Deflection [μm]'] = result_values_microns
        self.model.data = df.to_dict(orient='index')
    
        result_values = [resultu1, resultu2, resultu3, resultu4, resultu5, resultu6, resultu7]
        df.loc[:len(result_values)-1, 'Deflection [mm]'] = result_values
        self.model.data = df.to_dict(orient='index')
        
        # **Update colors after computation**
        self.highlight_results()
        
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Compute Finished !")

        self.result_text.insert(tk.END, f"Compute Finished !")

    def highlight_results(self):
        """**Highlight computed results with different color**"""
        computed_color = '#90EE90'  # Light green for computed results
        deflection_col = self.get_column_index('Deflection [μm]')
        
        if deflection_col is not None:
            for row in range(7):
                # **Check if there's a computed value**
                cell_value = str(self.model.data.get(row, {}).get('Deflection [μm]', ''))
                if cell_value not in ['', 'nan', 'None'] and cell_value.replace('.', '').replace('-', '').isdigit():
                    self.table.setcellColor(row, deflection_col, computed_color)
        
        self.table.redraw()

if __name__ == '__main__':
    root = tk.Tk()
    root.geometry("1280x600")
    root.title("WuWan v0.2")
    app = TableApp(root)
    
    root.mainloop()


TypeError: 'int' object is not iterable

In [ ]:
class TableApp:
    def __init__(self, root):
        self.frame = ttk.Frame(root)
        self.frame.pack(fill=tk.BOTH, expand=True)

        # Create a TableModel and TableCanvas
        self.model = TableModel()
        self.table = TableCanvas(self.frame, model=self.model, rowheight=30)
        self.table.show()

        # Load initial data
        self.load_data()

        # Adjust column widths
        self.adjust_column_widths()

        # Create a frame for the button to center it
        button_frame = ttk.Frame(root)
        button_frame.pack(pady=10)
        
        # Only keep the Compute Deflections button
        self.get_data_button1 = tk.Button(
            button_frame, 
            text="Compute Deflections", 
            command=self.get_data,
            font=("Arial", 10, "bold"),
            bg="#4CAF50",
            fg="white",
            padx=20,
            pady=5
        )
        self.get_data_button1.pack()

        # Create the result Text widget
        self.result_text = tk.Text(root, height=5, width=200)
        self.result_text.pack(pady=10)

    def load_data(self):
        # Sample pandas DataFrame
        data = {
            'Layer': ['layer 1', 'layer 2', 'layer 3', 'layer 4', 'layer 5', '', ''],
            'Modulus [MPa]': [4000, 400, 300, 200, 100, '', ''],
            'Poisson [-]': ['0.3', '0.35', '0.35', '0.4', '0.4', '', ''],
            'Thickness [mm]': ['150', '240', '300', '500', 'inf', '', ''],
            'Load': ['load_1', '', '', '', '', '', ''],
            'Stress [MPa]': ['0.95', '', '', '', '', '', ''],
            'Radius [mm]': ['150', '', '', '', '', '', ''],
            'Evaluation point': ['point 1', 'point 2', 'point 3', 'point 4', 'point 5', 'point 6', 'point 7'],
            'r [mm]': ['0', '300', '600', '900', '1200', '1500', '1800'],
            'Deflection [μm]': ['', '', '', '', '', '', ''],
            'perfect: 1, no: 0': ['1', '', '', '', '', '', ''],
        }
        df = pd.DataFrame(data)

        # Clear the model's data and column names
        self.model.data = {}
        self.model.columnNames = []
        self.model.reclist = []

        # Convert DataFrame to dict and import to TableModel
        data_dict = df.to_dict(orient='index')
        self.model.importDict(data_dict)
        self.table.redraw()

    def adjust_column_widths(self):
        # Adjust the column widths based on column headers and content length
        for col_name in self.model.columnNames:
            self.model.columnwidths[col_name] = max(len(col_name) * 10, 80)
        self.table.redraw()

    def get_data(self):
        # Access the data from the model
        data = self.model.data
              
        # Convert it back to pandas DataFrame
        df = pd.DataFrame.from_dict(data, orient='index')

        # Save
        df.to_csv('data.csv', index=False)
        
        # Run compute module
        df_numeric = df.apply(pd.to_numeric, errors='coerce')
        df_numeric_filled = df_numeric.fillna(0).astype(float)
        Data_array = df_numeric_filled.to_numpy()
        start_time = time.time()
        
        resultz, resultu1, resultu2, resultu3, resultu4, resultu5, resultu6, resultu7 = Calculation(Data_array)

        result_values = [resultu1, resultu2, resultu3, resultu4, resultu5, resultu6, resultu7]
        result_values_microns = [round(value * 1000, 3) for value in result_values]
    
        
        df.loc[:len(result_values_microns)-1, 'Deflection [μm]'] = result_values_microns
        self.model.data = df.to_dict(orient='index')
    

        result_values = [resultu1, resultu2, resultu3, resultu4, resultu5, resultu6, resultu7]
        df.loc[:len(result_values)-1, 'Deflection [mm]'] = result_values
        self.model.data = df.to_dict(orient='index')
        
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Compute Finished !")

        self.result_text.insert(tk.END, f"Compute Finished !")

if __name__ == '__main__':
    root = tk.Tk()
    root.geometry("1280x600")
    root.title("WuWan v0.2")
    app = TableApp(root)
    
    root.mainloop()


In [ ]:
class TableApp:
    def __init__(self, root):
        self.frame = ttk.Frame(root)
        self.frame.pack(fill=tk.BOTH, expand=True)

        # Create a TableModel and TableCanvas
        self.model = TableModel()
        self.table = TableCanvas(self.frame, model=self.model, rowheight=30)
        self.table.show()

        # Load initial data
        self.load_data()

        # Adjust column widths
        self.adjust_column_widths()

        # Button to get updated data
        self.get_data_button1 = tk.Button(root, text="Compute Deflections", command=self.get_data)
        self.get_data_button2 = tk.Button(root, text="FWD Backcalculation", command=self.get_data_measurement)
        self.get_data_button3 = tk.Button(root, text="switch to Calculation", command=self.switch_to_compute)
        self.get_data_button4 = tk.Button(root, text="switch to Backcalculation", command=self.switch_to_FWD)
        self.get_data_button1.pack(pady=1)
        self.get_data_button2.pack(pady=1)
        self.get_data_button3.pack(pady=1)
        self.get_data_button4.pack(pady=1)

        self.get_data_button3.pack(side=tk.LEFT, padx=15)
        self.get_data_button1.pack(side=tk.LEFT, padx=15)

        self.get_data_button4.pack(side=tk.RIGHT, padx=15)
        self.get_data_button2.pack(side=tk.RIGHT, padx=15)


        # **Create the result Text widget once in the __init__ method**
        self.result_text = tk.Text(root, height=5, width=200)
        self.result_text.pack(pady=10)

    def load_data(self):
        # Sample pandas DataFrame
        data = {
            'Layer': ['1', '2', '3', '4', '5', '6', '7'],
            'Modulus [MPa]': [4000, 400, 300, 200, 100, '', ''],
            'Poisson [-]': ['0.3', '0.35', '0.35', '0.4', '0.4', '', ''],
            'Thickness [mm]': ['150', '240', '300', '500', 'inf', '', ''],
            'Load': ['load_1', '', '', '', '', '', ''],
            'Stress [MPa]': ['0.95', '', '', '', '', '', ''],
            'Radius [mm]': ['150', '', '', '', '', '', ''],
            'Evaluation point': ['point 1', 'point 2', 'point 3', 'point 4', 'point 5', 'point 6', 'point 7'],
            'r [mm]': ['0', '300', '600', '900', '1200', '1500', '1800'],
            'Deflection [mm]': ['', '', '', '', '', '', ''],
            'perfect: 1, no: 0': ['1', '', '', '', '', '', ''],
        }
        df = pd.DataFrame(data)

        # **Clear the model's dx`ata and column names**
        self.model.data = {}
        self.model.columnNames = []
        self.model.reclist = []

        # Convert DataFrame to dict and import to TableModel
        data_dict = df.to_dict(orient='index')
        self.model.importDict(data_dict)
        self.table.redraw()

    def load_data_measurement(self):
        
        # Sample pandas DataFrame
        data = {
            'Layer': ['1', '2', '3', '4', '5', '', ''],
            'Modulus [MPa]': ['x', 'x', 'x', 'x', 'x', '', ''],
            'Poisson [-]': ['0.3', '0.35', '0.35', '0.4', '0.4', '', ''],
            'Thickness [mm]': ['150', '240', '300', 'x', 'inf', '', ''],
            'Load': ['load_1', '', '', '', '', '', ''],
            'Stress [MPa]': ['0.95', '', '', '', '', '', ''],
            'Radius [mm]': ['150', '', '', '', '', '', ''],
            'Evaluation point': ['Evaluation point', '', '', '', '', '', ''],
            'r [mm]': ['0', '300', '600', '900', '1200', '1500', '1800'],
            'z [mm]': ['0', '0', '0', '0', '0', '0', '0'],
            'u_z (measured) [micron]': ['555', '368', '231', '152', '105', '75', '57'],
        }
        df = pd.DataFrame(data)

        
        # Convert DataFrame to dict and import to TableModel
        data_dict = df.to_dict(orient='index')
        self.model.importDict(data_dict)
        

        self.table.redraw()

    def adjust_column_widths(self):
        # Adjust the column widths based on column headers and content length
        # Set an appropriate width for each column name
        for col_name in self.model.columnNames:
            self.model.columnwidths[col_name] = max(len(col_name) * 10, 80)

        self.table.redraw()

    def get_data(self):

        # Access the data from the model
        data = self.model.data
              
        # Convert it back to pandas DataFrame
        df = pd.DataFrame.from_dict(data, orient='index')

        # Save
        df.to_csv('data.csv', index=False)
        
        # run compute module
        #%run Compute_1.ipynb
        df_numeric = df.apply(pd.to_numeric, errors='coerce')
        df_numeric_filled = df_numeric.fillna(0).astype(float)
        Data_array = df_numeric_filled.to_numpy()
        start_time = time.time()
        #profiler = cProfile.Profile()
        #profiler.enable()
        #resultz, resultu = Calculation(df)
        resultz, resultu1, resultu2, resultu3, resultu4, resultu5, resultu6, resultu7 = Calculation(Data_array)

        result_values = [resultu1, resultu2, resultu3, resultu4, resultu5, resultu6, resultu7]
        df.loc[:len(result_values)-1, 'Deflection [mm]'] = result_values
        self.model.data = df.to_dict(orient='index')
        
        #self.refresh_table()

        #indices = np.arange(len(interval_value))

        #plt.figure(figsize=(10, 6))
        #plt.plot(indices[0:200], interval_value[0:200], 'b-', linewidth=2, label='Interval Values')
        #plt.xlabel('**Index**')
        #plt.ylabel('**Interval Value**')
        #plt.title('**Interval Value Distribution**')
        #plt.grid(True, alpha=0.3)
        #plt.legend()
        #plt.show()
        #profiler.disable()
        #stats = pstats.Stats(profiler)
        #stats.sort_stats('cumulative').print_stats(20)  
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"执行时间: {execution_time} 秒")
        

        # **Clear previous content of the Text widget**
        self.result_text.delete('1.0', tk.END)

        # **Insert new result into the Text widget**
        self.result_text.insert(tk.END, f"sigma_z: {resultz} MPa; u_z: {resultu1} mm\n")

        # Print or process the DataFrame (optional)

        
    def get_data_measurement(self):
        
        # Access the data from the model
        data = self.model.data
              
        # Convert it back to pandas DataFrame
        df = pd.DataFrame.from_dict(data, orient='index')

        # Save
        df.to_csv('data.csv', index=False)
        
        # run compute module
        start_time = time.time()
        
        #profiler = cProfile.Profile()
        #profiler.enable()
        #Back_Calculation(df)
        
        df, cost_function = Back_Calculation(df)
        
        end_time = time.time()
        execution_time = end_time - start_time
        #profiler.disable()
        #stats = pstats.Stats(profiler)
        #stats.sort_stats('cumulative').print_stats(20)  # 显示前20个最耗时的调用

        print(f"run time: {execution_time} sec")
        
        print(df)
        

        # **Clear previous content of the Text widget**
        self.result_text.delete('1.0', tk.END)

        # **Insert new result into the Text widget**
        self.result_text.insert(tk.END, f"cost_function: {cost_function}\n")

        # Convert DataFrame to dict and import to TableModel
        data_dict = df.to_dict(orient='index')
        self.model.importDict(data_dict)
        self.table.redraw()

        
    def switch_to_compute(self):
        
        # **Clear the model's data and column names**
        self.model.data = {}
        self.model.columnNames = []
        self.model.reclist = []
        
        # Load initial data
        self.load_data()

        # Adjust column widths
        self.adjust_column_widths()
        
        # Access the data from the model
        data = self.model.data
              
        # Convert it back to pandas DataFrame
        df = pd.DataFrame.from_dict(data, orient='index')

        # Save
        df.to_csv('data.csv', index=False)


    def switch_to_FWD(self):
        
        # **Clear the model's data and column names**
        self.model.data = {}
        self.model.columnNames = []
        self.model.reclist = []
        
        # Load initial data
        self.load_data_measurement()

        # Adjust column widths
        self.adjust_column_widths()
        
        # Access the data from the model
        data = self.model.data
              
        # Convert it back to pandas DataFrame
        df = pd.DataFrame.from_dict(data, orient='index')

        # Save
        df.to_csv('data.csv', index=False)


if __name__ == '__main__':
    root = tk.Tk()
    root.geometry("1280x400")
    root.title("WuWan v0.2")
    app = TableApp(root)
    
    root.mainloop()
    

# test_profile.py
import cProfile
import pstats
from compute_cython import Calculation
import numpy as np
import pandas as pd
data = {
        'Layer': ['1', '2', '3', '4', '5'],
        'Modulus [MPa]': [4000, 400, 300, 200, 100],
        'Poisson [-]': ['0.3', '0.35', '0.35', '0.4', '0.4'],
        'Thickness [mm]': ['150', '240', '300', '500', 'inf'],
        'Load': ['load_1', '', '', '', ''],
        'Stress [MPa]': ['0.95', '', '', '', ''],
        'Radius [mm]': ['150', '', '', '', ''],
        'Evaluation point': ['Evaluation point', '', '', '', ''],
        'r [mm]': ['0', '', '', '', ''],
        'z [mm]': ['0', '', '', '', ''],
        'perfect: 1, no: 0': ['1', '', '', '', ''],
    }
df = pd.DataFrame(data)
df_numeric = df.apply(pd.to_numeric, errors='coerce')
df_numeric_filled = df_numeric.fillna(0).astype(float)
Data_array = df_numeric_filled.to_numpy()
# 创建性能分析器
profiler = cProfile.Profile()

# 开始分析
profiler.enable()

# 运行测试函数
resultz, resultu = Calculation(Data_array)

# 停止分析
profiler.disable()

# 创建统计对象并打印结果
stats = pstats.Stats(profiler)
stats.sort_stats('cumulative')  # 按累计时间排序
stats.print_stats()

# 也可以只显示前10行
# stats.print_stats(10)

# 保存到文件
stats.dump_stats("fibonacci_profile.prof")


In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import pandas as pd
import time

class LightweightTableApp:
    def __init__(self, root):
        self.root = root
        self.df = pd.DataFrame()
        self.setup_ui()
        self.initialize_table()
        self.current_mode = "compute"
        
    def setup_ui(self):
        """**设置UI布局**"""
        # **创建主要容器**
        main_frame = ttk.Frame(self.root, padding="10")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # **工具栏**
        toolbar = ttk.Frame(main_frame)
        toolbar.pack(fill=tk.X, pady=(0, 10))
        
        # **模式切换按钮**
        mode_frame = ttk.LabelFrame(toolbar, text="Mode Selection", padding="5")
        mode_frame.pack(side=tk.LEFT, padx=(0, 10))
        
        ttk.Button(mode_frame, text="📊 Calculation", 
                  command=self.switch_to_compute, width=15).pack(side=tk.LEFT, padx=2)
        ttk.Button(mode_frame, text="🔬 Backcalculation", 
                  command=self.switch_to_FWD, width=15).pack(side=tk.LEFT, padx=2)
        
        # **操作按钮**
        action_frame = ttk.LabelFrame(toolbar, text="Actions", padding="5")
        action_frame.pack(side=tk.LEFT, padx=(0, 10))
        
        self.calc_button = ttk.Button(action_frame, text="▶ Run Calculation", 
                                     command=self.get_data, width=15)
        self.calc_button.pack(side=tk.LEFT, padx=2)
        
        self.back_calc_button = ttk.Button(action_frame, text="▶ Run Backcalc", 
                                          command=self.get_data_measurement, width=15)
        self.back_calc_button.pack(side=tk.LEFT, padx=2)
        
        # **文件操作**
        file_frame = ttk.LabelFrame(toolbar, text="File Operations", padding="5")
        file_frame.pack(side=tk.RIGHT)
        
        ttk.Button(file_frame, text="💾 Save", 
                  command=self.save_data, width=10).pack(side=tk.LEFT, padx=2)
        ttk.Button(file_frame, text="📁 Load", 
                  command=self.load_from_file, width=10).pack(side=tk.LEFT, padx=2)
        
        # **表格区域**
        self.create_table(main_frame)
        
        # **状态和结果区域**
        self.create_results_area(main_frame)

    def create_table(self, parent):
        """**创建表格组件**"""
        # **表格容器**
        table_frame = ttk.LabelFrame(parent, text="📋 Data Table", padding="5")
        table_frame.pack(fill=tk.BOTH, expand=True, pady=(0, 10))
        
        # **创建Treeview**
        self.tree = ttk.Treeview(table_frame)
        
        # **滚动条**
        v_scroll = ttk.Scrollbar(table_frame, orient=tk.VERTICAL, command=self.tree.yview)
        h_scroll = ttk.Scrollbar(table_frame, orient=tk.HORIZONTAL, command=self.tree.xview)
        
        self.tree.configure(yscrollcommand=v_scroll.set, xscrollcommand=h_scroll.set)
        
        # **布局**
        self.tree.grid(row=0, column=0, sticky='nsew')
        v_scroll.grid(row=0, column=1, sticky='ns')
        h_scroll.grid(row=1, column=0, sticky='ew')
        
        table_frame.grid_rowconfigure(0, weight=1)
        table_frame.grid_columnconfigure(0, weight=1)
        
        # **双击编辑**
        self.tree.bind('<Double-1>', self.on_double_click)
        
        # **添加行按钮**
        button_frame = ttk.Frame(table_frame)
        button_frame.grid(row=2, column=0, columnspan=2, sticky='ew', pady=(5, 0))
        
        ttk.Button(button_frame, text="➕ Add Row", 
                  command=self.add_row).pack(side=tk.LEFT, padx=2)
        ttk.Button(button_frame, text="➖ Delete Row", 
                  command=self.delete_row).pack(side=tk.LEFT, padx=2)

    def create_results_area(self, parent):
        """**创建结果显示区域**"""
        results_frame = ttk.LabelFrame(parent, text="📊 Results & Status", padding="5")
        results_frame.pack(fill=tk.X, pady=(10, 0))
        
        # **状态栏**
        status_frame = ttk.Frame(results_frame)
        status_frame.pack(fill=tk.X, pady=(0, 5))
        
        ttk.Label(status_frame, text="Status:").pack(side=tk.LEFT)
        self.status_label = ttk.Label(status_frame, text="Ready", 
                                     foreground="green", font=('Arial', 9, 'bold'))
        self.status_label.pack(side=tk.LEFT, padx=(5, 0))
        
        # **结果文本区域**
        text_frame = ttk.Frame(results_frame)
        text_frame.pack(fill=tk.BOTH, expand=True)
        
        self.result_text = tk.Text(text_frame, height=5, font=('Consolas', 9), 
                                  wrap=tk.WORD, state=tk.DISABLED)
        
        result_scroll = ttk.Scrollbar(text_frame, orient=tk.VERTICAL, 
                                     command=self.result_text.yview)
        self.result_text.configure(yscrollcommand=result_scroll.set)
        
        self.result_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        result_scroll.pack(side=tk.RIGHT, fill=tk.Y)

    def on_double_click(self, event):
        """**处理双击编辑事件**"""
        selection = self.tree.selection()
        if not selection:
            return
        
        item = selection[0]
        column = self.tree.identify_column(event.x)
        
        if column:
            self.edit_cell(item, column)

    def edit_cell(self, item, column):
        """**编辑单元格**"""
        col_index = int(column.replace('#', '')) - 1
        if col_index < 0:
            return
            
        # **获取当前值**
        values = list(self.tree.item(item)['values'])
        if col_index >= len(values):
            return
            
        current_value = values[col_index]
        
        # **创建编辑框**
        bbox = self.tree.bbox(item, column)
        if not bbox:
            return
            
        entry = tk.Entry(self.tree, font=('Arial', 10))
        entry.insert(0, str(current_value))
        entry.select_range(0, tk.END)
        entry.place(x=bbox[0], y=bbox[1], width=bbox[2], height=bbox[3])
        
        def save_edit(event=None):
            new_value = entry.get()
            values[col_index] = new_value
            self.tree.item(item, values=values)
            entry.destroy()
            self.update_dataframe()
            
        def cancel_edit(event=None):
            entry.destroy()
            
        entry.bind('<Return>', save_edit)
        entry.bind('<Escape>', cancel_edit)
        entry.bind('<FocusOut>', save_edit)
        entry.focus()

    def update_dataframe(self):
        """**从Treeview更新DataFrame**"""
        data = []
        columns = [self.tree.heading(col)['text'] for col in self.tree['columns']]
        
        for item in self.tree.get_children():
            values = self.tree.item(item)['values']
            data.append(values)
        
        if data and columns:
            self.df = pd.DataFrame(data, columns=columns)

    def load_dataframe_to_tree(self, df):
        """**将DataFrame加载到Treeview**"""
        # **清除现有数据**
        for item in self.tree.get_children():
            self.tree.delete(item)
        
        if df.empty:
            return
            
        # **设置列**
        columns = list(df.columns)
        self.tree['columns'] = columns
        self.tree['show'] = 'headings'
        
        # **配置列标题和宽度**
        for col in columns:
            self.tree.heading(col, text=col)
            # **根据内容调整列宽**
            max_width = max(len(str(col)) * 10, 100)
            if not df[col].empty:
                content_width = max(df[col].astype(str).str.len()) * 8
                max_width = max(max_width, content_width, 80)
            self.tree.column(col, width=min(max_width, 200), minwidth=80)
        
        # **插入数据**
        for index, row in df.iterrows():
            self.tree.insert('', tk.END, values=list(row))

    def initialize_table(self):
        """**初始化表格**"""
        self.load_data()
        self.update_button_states()

    def load_data(self):
        """**加载计算模式数据**"""
        data = {
            'Layer': ['1', '2', '3', '4', '5'],
            'Modulus [MPa]': [4000, 400, 300, 200, 100],
            'Poisson [-]': [0.3, 0.35, 0.35, 0.4, 0.4],
            'Thickness [mm]': [150, 240, 300, 500, 'inf'],
            'Load': ['load_1', '', '', '', ''],
            'Stress [MPa]': [0.95, '', '', '', ''],
            'Radius [mm]': [150, '', '', '', ''],
            'Evaluation point': ['Evaluation point', '', '', '', ''],
            'r [mm]': [0, '', '', '', ''],
            'z [mm]': [0, '', '', '', ''],
            'perfect: 1, no: 0': [1, '', '', '', ''],
        }
        self.df = pd.DataFrame(data)
        self.load_dataframe_to_tree(self.df)
        self.current_mode = "compute"
        self.status_label.config(text="Calculation Mode", foreground="blue")

    def load_data_measurement(self):
        """**加载测量模式数据**"""
        data = {
            'Layer': ['1', '2', '3', '4', '5', '', ''],
            'Modulus [MPa]': ['x', 'x', 'x', 'x', 'x', '', ''],
            'Poisson [-]': [0.3, 0.35, 0.35, 0.4, 0.4, '', ''],
            'Thickness [mm]': [150, 240, 300, 'x', 'inf', '', ''],
            'Load': ['load_1', '', '', '', '', '', ''],
            'Stress [MPa]': [0.95, '', '', '', '', '', ''],
            'Radius [mm]': [150, '', '', '', '', '', ''],
            'Evaluation point': ['Evaluation point', '', '', '', '', '', ''],
            'r [mm]': [0, 300, 600, 900, 1200, 1500, 1800],
            'z [mm]': [0, 0, 0, 0, 0, 0, 0],
            'u_z (measured) [micron]': [555, 368, 231, 152, 105, 75, 57],
        }
        self.df = pd.DataFrame(data)
        self.load_dataframe_to_tree(self.df)
        self.current_mode = "measurement"
        self.status_label.config(text="Backcalculation Mode", foreground="purple")

    def add_row(self):
        """**添加新行**"""
        if not self.df.empty:
            new_row = [''] * len(self.df.columns)
            self.tree.insert('', tk.END, values=new_row)
            self.update_dataframe()

    def delete_row(self):
        """**删除选中行**"""
        selection = self.tree.selection()
        if selection:
            for item in selection:
                self.tree.delete(item)
            self.update_dataframe()

    def save_data(self):
        """**保存数据到CSV**"""
        try:
            self.update_dataframe()
            filename = filedialog.asksaveasfilename(
                defaultextension=".csv",
                filetypes=[("CSV files", "*.csv"), ("All files", "*.*")]
            )
            if filename:
                self.df.to_csv(filename, index=False)
                self.display_result(f"✅ Data saved to {filename}\n")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save data: {str(e)}")

    def load_from_file(self):
        """**从文件加载数据**"""
        try:
            filename = filedialog.askopenfilename(
                filetypes=[("CSV files", "*.csv"), ("Excel files", "*.xlsx"), ("All files", "*.*")]
            )
            if filename:
                if filename.endswith('.csv'):
                    self.df = pd.read_csv(filename)
                elif filename.endswith('.xlsx'):
                    self.df = pd.read_excel(filename)
                else:
                    return
                    
                self.load_dataframe_to_tree(self.df)
                self.display_result(f"✅ Data loaded from {filename}\n")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load data: {str(e)}")

    def update_button_states(self):
        """**更新按钮状态**"""
        if self.current_mode == "compute":
            self.calc_button.config(state="normal")
            self.back_calc_button.config(state="disabled")
        else:
            self.calc_button.config(state="disabled")
            self.back_calc_button.config(state="normal")

    def get_data(self):
        """**执行计算**"""
        try:
            self.status_label.config(text="Computing...", foreground="orange")
            self.root.update()
            
            self.update_dataframe()
            self.df.to_csv('data.csv', index=False)
            df_numeric = self.df.apply(pd.to_numeric, errors='coerce')
            df_numeric_filled = df_numeric.fillna(0).astype(float)
            Data_array = df_numeric_filled.to_numpy()
            
            start_time = time.time()
            # **这里调用你的计算函数**
            resultz, resultu = Calculation(Data_array)
            
            # **模拟计算**
            #resultz, resultu = 1.234, 0.567
            execution_time = time.time() - start_time
            
            result_text = (
                f"✅ Calculation completed in {execution_time:.3f}s\n"
                f"📊 Vertical Stress (σz): {resultz:.4f} MPa\n"
                f"📊 Vertical Displacement (uz): {resultu:.4f} mm\n"
                f"{'-'*50}\n"
            )
            
            self.display_result(result_text)
            self.status_label.config(text="Calculation Complete", foreground="green")
            
        except Exception as e:
            error_msg = f"❌ Calculation failed: {str(e)}\n"
            self.display_result(error_msg)
            self.status_label.config(text="Calculation Failed", foreground="red")

    def get_data_measurement(self):
        """**执行反算**"""
        try:
            self.status_label.config(text="Running Backcalculation...", foreground="orange")
            self.root.update()
            
            self.update_dataframe()
            self.df.to_csv('data.csv', index=False)
            
            start_time = time.time()
            # **这里调用你的反算函数**
            # df, cost_function = Back_Calculation(df)
            
            # **模拟反算**
            cost_function = 0.00123
            execution_time = time.time() - start_time
            
            result_text = (
                f"✅ Backcalculation completed in {execution_time:.3f}s\n"
                f"🎯 Cost function: {cost_function:.6f}\n"
                f"{'-'*50}\n"
            )
            
            self.display_result(result_text)
            self.status_label.config(text="Backcalculation Complete", foreground="green")
            
        except Exception as e:
            error_msg = f"❌ Backcalculation failed: {str(e)}\n"
            self.display_result(error_msg)
            self.status_label.config(text="Backcalculation Failed", foreground="red")

    def switch_to_compute(self):
        """**切换到计算模式**"""
        self.load_data()
        self.update_button_states()

    def switch_to_FWD(self):
        """**切换到反算模式**"""
        self.load_data_measurement()
        self.update_button_states()

    def display_result(self, text):
        """**显示结果**"""
        self.result_text.config(state=tk.NORMAL)
        self.result_text.insert(tk.END, text)
        self.result_text.see(tk.END)
        self.result_text.config(state=tk.DISABLED)


def main():
    """**主函数**"""
    root = tk.Tk()
    root.geometry("1200x700")
    root.title("🔬 Lightweight Pavement Analysis Tool")
    root.minsize(1000, 600)
    
    # **设置样式**
    try:
        style = ttk.Style()
        available_themes = style.theme_names()
        if 'clam' in available_themes:
            style.theme_use('clam')
        elif 'alt' in available_themes:
            style.theme_use('alt')
    except:
        pass
    
    app = LightweightTableApp(root)
    
    # **快捷键**
    root.bind('<Control-s>', lambda e: app.save_data())
    root.bind('<Control-o>', lambda e: app.load_from_file())
    root.bind('<Control-q>', lambda e: root.quit())
    root.bind('<F5>', lambda e: app.get_data() if app.current_mode == "compute" else app.get_data_measurement())
    root.bind('<Delete>', lambda e: app.delete_row())
    
    root.mainloop()


if __name__ == '__main__':
    main()
